<a href="https://colab.research.google.com/github/Lihini827/Facial-Expression-Recognition-1-/blob/main/needjaffecrossval(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,BatchNormalization,Dropout

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
#datasetFolderName=root_path+'DS'
#MODEL_FILENAME=root_path+"model_cv.h5"
sourceFiles=[]
classLabels=['ANGRY','DISGUST', 'FEAR','HAPPY','NEUTRAL','SAD','SUPRISE']
X=[]
Y=[]

img_rows, img_cols =  224, 224 # input image dimensions
train_path='/content/drive/MyDrive/Facial_Expression/dataset/datasetJAFFEtrain,test,validationwithout aug/train/'
validation_path='/content/drive/MyDrive/Facial_Expression/dataset/datasetJAFFEtrain,test,validationwithout aug/validation/'
test_path='/content/drive/MyDrive/Facial_Expression/dataset/datasetJAFFEtrain,test,validationwithout aug/test/'

In [ ]:
datasetFolderName='/content/drive/MyDrive/Facial_Expression/dataset/datasetJAFFEtrain,test,validationwithout aug'

In [ ]:
def transferBetweenFolders(source, dest, splitRate):
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")

def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/',
                               datasetFolderName+'/'+dest+'/'+label+'/',
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score

In [ ]:
transferAllClassBetweenFolders('test', 'train', 1.0)

In [ ]:
transferAllClassBetweenFolders('train', 'test', 0.20)

In [ ]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
          if(folderName==classLabels[i]):
              Y.append(i)

In [ ]:
for i in range(len(classLabels)):
  prepareNameWithLabels(classLabels[i])

In [ ]:
X=np.asarray(X)
Y=np.asarray(Y)

In [ ]:
seed_value= 12321
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)
# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input

In [ ]:
batch_size = 16
epoch=50
IMAGE_SIZE = [224, 224]

def getModel():
  import tensorflow
  densenet =tensorflow.keras.applications.DenseNet201(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
  x=densenet.output
  x= GlobalAveragePooling2D()(x)
  x= BatchNormalization()(x)
  x= Dropout(0.4,seed=seed_value)(x)
  x= Dense(1024,activation='relu')(x)
  x= Dense(128,activation='relu')(x)
  x= BatchNormalization()(x)
  x= Dropout(0.4, seed=seed_value)(x)

  preds=Dense(7,activation='softmax')(x)
  model=Model(inputs=densenet.input,outputs=preds)
  for layer in densenet.layers[:-8]:
    layer.trainable=False

  for layer in densenet.layers[-8:]:
    layer.trainable=True

  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

  return model




model=getModel()

74836368/74836368 [==============================] - 0s 0us/step


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex],
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      zoom_range=0.20,
                      fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)

    # fit model
    history=model.fit_generator(train_generator,
                        epochs=epoch)

    predictions = model.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

Results for fold 1
Found 148 images belonging to 7 classes.
Found 30 images belonging to 7 classes.


<ipython-input-14-6fd8d01f99b8>:43: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_generator,


Epoch 1/50
10/10 [==============================] - 65s 3s/step - loss: 2.8179 - accuracy: 0.2027
Epoch 2/50
10/10 [==============================] - 2s 221ms/step - loss: 2.1606 - accuracy: 0.3378
Epoch 3/50
10/10 [==============================] - 2s 245ms/step - loss: 1.6840 - accuracy: 0.4392
Epoch 4/50
10/10 [==============================] - 2s 216ms/step - loss: 1.5310 - accuracy: 0.4527
Epoch 5/50
10/10 [==============================] - 2s 239ms/step - loss: 1.2080 - accuracy: 0.5946
Epoch 6/50
10/10 [==============================] - 4s 438ms/step - loss: 1.1561 - accuracy: 0.6081
Epoch 7/50
10/10 [==============================] - 3s 269ms/step - loss: 1.1105 - accuracy: 0.6149
Epoch 8/50
10/10 [==============================] - 2s 222ms/step - loss: 1.1471 - accuracy: 0.6081
Epoch 9/50
10/10 [==============================] - 2s 213ms/step - loss: 0.9094 - accuracy: 0.7095
Epoch 10/50
10/10 [==============================] - 2s 242ms/step - loss: 0.7707 - accuracy: 0.7635
E

<ipython-input-14-6fd8d01f99b8>:46: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(validation_generator, verbose=1)


2/2 [==============================] - 10s 7s/step
***Performance on Validation data***
Accuracy  : 0.6333333333333333
Precision : 0.7154497354497354
f1Score : 0.6066522366522366
[[5 0 0 0 0 0 0]
 [1 2 1 0 0 0 0]
 [0 1 3 0 0 0 0]
 [0 0 0 4 0 0 0]
 [2 0 0 1 2 0 0]
 [1 0 1 1 0 1 0]
 [0 0 0 1 1 0 2]]
Results for fold 2
Found 148 images belonging to 7 classes.
Found 30 images belonging to 7 classes.


...

2/2 [==============================] - 3s 2s/step
***Performance on Validation data***
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[4 0 0 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 5 0 0 0]
 [0 0 0 0 4 0 0]
 [0 0 0 0 0 4 0]
 [0 0 0 0 0 0 4]]
Results for fold 5
Found 149 images belonging to 7 classes.
Found 29 images belonging to 7 classes.


<ipython-input-14-6fd8d01f99b8>:43: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_generator,


Epoch 1/50
10/10 [==============================] - 2s 220ms/step - loss: 0.1095 - accuracy: 0.9664
Epoch 2/50
10/10 [==============================] - 2s 218ms/step - loss: 0.1066 - accuracy: 0.9597
Epoch 3/50
10/10 [==============================] - 4s 358ms/step - loss: 0.1438 - accuracy: 0.9597
Epoch 4/50
10/10 [==============================] - 2s 217ms/step - loss: 0.1935 - accuracy: 0.9262
Epoch 5/50
10/10 [==============================] - 2s 214ms/step - loss: 0.1217 - accuracy: 0.9530
Epoch 6/50
10/10 [==============================] - 2s 234ms/step - loss: 0.1224 - accuracy: 0.9597
Epoch 7/50
10/10 [==============================] - 2s 215ms/step - loss: 0.1884 - accuracy: 0.9329
Epoch 8/50
10/10 [==============================] - 3s 259ms/step - loss: 0.0694 - accuracy: 0.9799
Epoch 9/50
10/10 [==============================] - 3s 232ms/step - loss: 0.1132 - accuracy: 0.9597
Epoch 10/50
10/10 [==============================] - 2s 213ms/step - loss: 0.0892 - accuracy: 0.9732

<ipython-input-14-6fd8d01f99b8>:46: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(validation_generator, verbose=1)


2/2 [==============================] - 0s 75ms/step
***Performance on Validation data***
Accuracy  : 0.8620689655172413
Precision : 0.9077586206896552
f1Score : 0.8595006526041009
[[4 0 0 0 0 0 0]
 [1 3 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 5 0 0 0]
 [0 0 0 1 3 0 0]
 [0 0 0 0 0 4 0]
 [0 0 0 2 0 0 2]]


In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)


==============TEST RESULTS============
Found 35 images belonging to 7 classes.
3/3 [==============================] - 8s 4s/step
Accuracy  : 0.9142857142857143
Precision : 0.9306122448979591
f1Score : 0.915873015873016
[[5 0 0 0 0 0 0]
 [0 4 0 0 0 1 0]
 [0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0]
 [0 0 0 1 4 0 0]
 [0 0 0 1 0 4 0]
 [0 0 0 0 0 0 5]]
